In [ ]:
import pandas as pd
file_path = 'group_stats.xlsx'
xls = pd.ExcelFile(file_path)
sheet_names = xls.sheet_names
sheet_names
# Load the first sheet to examine the data structure
df = xls.parse('fingerTapping_HbO')
df.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import xgboost as xgb

# Clean and reshape the DataFrame
df_clean = df[1:].copy()  # Skip the header row in the first row
df_clean.columns = ['PatientID', 'IsParkinson?', 'L_max', 'L_timeToMax', 'L_mean', 'L_AUC',
                    'R_max', 'R_timeToMax', 'R_mean', 'R_AUC']
df_clean = df_clean.dropna()

# Convert appropriate columns to numeric
for col in df_clean.columns[1:]:
    df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

df_clean = df_clean.dropna()

# Features and labels
X = df_clean.drop(columns=['PatientID', 'IsParkinson?'])
y = df_clean['IsParkinson?'].astype(int)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train XGBoost model
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
report = classification_report(y_test, y_pred, output_dict=True)

# Display classification report
pd.DataFrame(report).transpose()
